# Sparkify Project Workspace
This workspace contains a tiny subset (128MB) of the full dataset available (12GB). Feel free to use this workspace to build your project, or to explore a smaller subset with Spark before deploying your cluster on the cloud. Instructions for setting up your Spark cluster is included in the last lesson of the Extracurricular Spark Course content.

You can follow the steps below to guide your data analysis and model building portion of this project.

In [2]:
# import libraries
from pyspark.sql import SparkSession
from pyspark.sql import types 
import pyspark.sql.functions as F
from pyspark.sql.window import Window

In [3]:
# create a Spark session
spark = SparkSession.builder.appName('sparkify').getOrCreate()

# Load and Clean Dataset
In this workspace, the mini-dataset file is `mini_sparkify_event_data.json`. Load and clean the dataset, checking for invalid or missing data - for example, records without userids or sessionids. 

## Load Data

In [4]:
path = "mini_sparkify_event_data.json"
user_log = spark.read.json(path)

In [5]:
user_log.take(3)

[Row(artist='Martha Tilston', auth='Logged In', firstName='Colin', gender='M', itemInSession=50, lastName='Freeman', length=277.89016, level='paid', location='Bakersfield, CA', method='PUT', page='NextSong', registration=1538173362000, sessionId=29, song='Rockpools', status=200, ts=1538352117000, userAgent='Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) Gecko/20100101 Firefox/31.0', userId='30'),
 Row(artist='Five Iron Frenzy', auth='Logged In', firstName='Micah', gender='M', itemInSession=79, lastName='Long', length=236.09424, level='free', location='Boston-Cambridge-Newton, MA-NH', method='PUT', page='NextSong', registration=1538331630000, sessionId=8, song='Canada', status=200, ts=1538352180000, userAgent='"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/37.0.2062.103 Safari/537.36"', userId='9'),
 Row(artist='Adam Lambert', auth='Logged In', firstName='Colin', gender='M', itemInSession=51, lastName='Freeman', length=282.8273, level='paid', location='

## Data Types

In [6]:
user_log.take(1)

[Row(artist='Martha Tilston', auth='Logged In', firstName='Colin', gender='M', itemInSession=50, lastName='Freeman', length=277.89016, level='paid', location='Bakersfield, CA', method='PUT', page='NextSong', registration=1538173362000, sessionId=29, song='Rockpools', status=200, ts=1538352117000, userAgent='Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) Gecko/20100101 Firefox/31.0', userId='30')]

In [7]:
user_log.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [8]:
user_log.filter('userid not rlike "[0-9]+"').select('userid').dropDuplicates().collect()

[Row(userid='')]

In [9]:
user_log.filter(user_log.userId == "").select('auth').dropDuplicates().collect()

[Row(auth='Logged Out'), Row(auth='Guest')]

In [10]:
user_log = user_log.withColumn("userId", user_log["userId"].cast(types.IntegerType()))

All columns have appropriate data types, except the `userId` column. I convert the variable from string to integer. By doing so, empty strings (as used for events from unregistered users) will become missing values.

## Missing values

In [11]:
def count_missing_values(df):
    """ Counts missing values for each column of df """
    
    missings = df.select([
        F.count(F.when(F.isnull(col) | F.isnan(col), 1)).alias(col) for col in df.columns
    ])
    
    return missings

In [12]:
print(count_missing_values(user_log).collect())

[Row(artist=58392, auth=0, firstName=8346, gender=8346, itemInSession=0, lastName=8346, length=58392, level=0, location=8346, method=0, page=0, registration=8346, sessionId=0, song=58392, status=0, ts=0, userAgent=8346, userId=8346)]


In [13]:
cond1 = (user_log.userId.isNotNull())
print(count_missing_values(user_log.filter(cond1)).collect())

[Row(artist=50046, auth=0, firstName=0, gender=0, itemInSession=0, lastName=0, length=50046, level=0, location=0, method=0, page=0, registration=0, sessionId=0, song=50046, status=0, ts=0, userAgent=0, userId=0)]


In [14]:
cond2 = (user_log.page == "NextSong")
print(count_missing_values(user_log.filter(cond1 & cond2)).collect())

[Row(artist=0, auth=0, firstName=0, gender=0, itemInSession=0, lastName=0, length=0, level=0, location=0, method=0, page=0, registration=0, sessionId=0, song=0, status=0, ts=0, userAgent=0, userId=0)]


There seems to be no major issue with missing values. Missing values occur in user specific variables (`firstName`, `gender`, `lastName`, `location`, `registration`, `userAgent`) only for events of unregistered users (for which holds `userId == None`), as well as in song specific variables (`artist`, `length`, `song`) for not song-related events (for which holds `page != "NextSong"`). 

In [15]:
user_log = user_log.filter(user_log.userId.isNotNull())

Since we are only interested in the behavior of registered users for the purpose of churn prediction, I will drop events of unregistered users for the further analysis.

# Exploratory Data Analysis
When you're working with the full dataset, perform EDA by loading a small subset of the data and doing basic manipulations within Spark. In this workspace, you are already provided a small subset of data you can explore.

### Define Churn

Once you've done some preliminary analysis, create a column `Churn` to use as the label for your model. I suggest using the `Cancellation Confirmation` events to define your churn, which happen for both paid and free users. As a bonus task, you can also look into the `Downgrade` events.

### Explore Data
Once you've defined churn, perform some exploratory data analysis to observe the behavior for users who stayed vs users who churned. You can start by exploring aggregates on these two groups of users, observing how much of a specific action they experienced per a certain time unit or number of songs played.

In [17]:
# Mark churn events
user_log = user_log.withColumn('churn', F.when( F.col('page') == "Cancellation Confirmation", 1 ).otherwise(0))

# Mark churn users
window_var = Window().partitionBy('userId')
user_log = user_log.withColumn('churn', F.max( F.col('churn') ).over(window_var))

In [18]:
user_log.groupBy('churn').count().show()

+-----+------+
|churn| count|
+-----+------+
|    1| 44864|
|    0|233290|
+-----+------+



In [19]:
user_log.groupBy('churn').agg(F.countDistinct('userId')).show()

+-----+----------------------+
|churn|count(DISTINCT userId)|
+-----+----------------------+
|    1|                    52|
|    0|                   173|
+-----+----------------------+



# Feature Engineering
Once you've familiarized yourself with the data, build out the features you find promising to train your model on. To work with the full dataset, you can follow the following steps.
- Write a script to extract the necessary features from the smaller subset of data
- Ensure that your script is scalable, using the best practices discussed in Lesson 3
- Try your script on the full data set, debugging your script if necessary

If you are working in the classroom workspace, you can just extract features based on the small subset of data contained here. Be sure to transfer over this work to the larger dataset when you work on your Spark cluster.

# Modeling
Split the full dataset into train, test, and validation sets. Test out several of the machine learning methods you learned. Evaluate the accuracy of the various models, tuning parameters as necessary. Determine your winning model based on test accuracy and report results on the validation set. Since the churned users are a fairly small subset, I suggest using F1 score as the metric to optimize.

# Final Steps
Clean up your code, adding comments and renaming variables to make the code easier to read and maintain. Refer to the Spark Project Overview page and Data Scientist Capstone Project Rubric to make sure you are including all components of the capstone project and meet all expectations. Remember, this includes thorough documentation in a README file in a Github repository, as well as a web app or blog post.